# Reading our wiki files, that we got from wikiextractor

### Downloading the libraries, that we will need

In [1]:
!pip install nltk

In [2]:
!pip install glob

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [ ]:
!pip install 

### Importing

In [2]:
# to process files
import glob
# to process regular expressions
import re
# to process texts
import nltk
# to numerate documents
from uuid import uuid4
# to see how the progress of cycles goes
from tqdm import tqdm
# main libraries
import pandas as pd
import numpy as np

In [38]:
# to use sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\1\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [62]:
# rules how to tokenize in Japanese
from nltk.tokenize import RegexpTokenizer
japanese_tokenizer = nltk.RegexpTokenizer(u'[^！？。]*[！？。]')

In [4]:
all_wikis_paths = []
for one_wiki_path in glob.iglob('data/*/*'):
    all_wikis_paths.append(one_wiki_path)

Reference: https://docs-python.ru/standart-library/modul-re-python/funktsija-escape-modulja-re/, https://stackoverflow.com/questions/5177850/correct-regexp-for-japanese-sentence-tokenizer-python

In [72]:
def separating_each_article(all_arts: str, divide_starting_by='<doc id='):
    all_articles = []
    # escape shields special symbols
    all_arts = re.split('(%s)' % re.escape(divide_starting_by), all_arts)
    for each_art in all_arts:
        if each_art != divide_starting_by and each_art != '':
            all_articles.append(divide_starting_by + each_art)
    return all_articles

def cleaning_special_chars(one_article, chars_to_clean=['\n', '\xa0']):
    # cleaning from html tags
    # all html tags with all possible content surrounded by <>
    cleaner = re.compile('<.*?>')
    cleaner = re.sub(cleaner, '', one_article)
    for one_char in chars_to_clean:
        cleaner = cleaner.replace(one_char, '')
    return cleaner

In [75]:
our_dataframe_wiki = pd.DataFrame()
for file in tqdm(all_wikis_paths):
    with open(file, encoding='utf-8') as f:
        whole_text = f.read()
    arts = separating_each_article(whole_text)
    our_df = pd.DataFrame(columns=['article_name', 'content'])
    # now we divide our articles by number and content
    for one_art in arts:
        to_num = uuid4()
        cleaned_article = cleaning_special_chars(one_art)
        tokenized_sent = japanese_tokenizer.tokenize(cleaned_article)
        our_df = our_df.append(pd.DataFrame({'article_name': [to_num] * len(tokenized_sent), 
                                            'content': tokenized_sent}))
    our_dataframe_wiki = pd.concat([our_dataframe_wiki, our_df])


  9%|▉         | 303/3433 [05:17<16:13,  3.21it/s]


 18%|█▊        | 605/3433 [07:46<38:19,  1.23it/s]


 26%|██▋       | 907/3433 [11:40<33:40,  1.25it/s]


 35%|███▌      | 1205/3433 [16:09<36:42,  1.01it/s]


 44%|████▎     | 1501/3433 [21:41<39:13,  1.22s/it]


...

 69%|██████▉   | 2384/3433 [49:08<52:42,  3.01s/it]


 78%|███████▊  | 2677/3433 [1:04:25<46:27,  3.69s/it]


 86%|████████▋ | 2963/3433 [1:22:28<35:16,  4.50s/it]


 95%|█████████▍| 3249/3433 [1:41:27<13:32,  4.41s/it]


100%|██████████| 3433/3433 [1:54:12<00:00,  2.00s/it]


In [78]:
our_dataframe_wiki

,article_name,content
0,beb8981c-a14b-4aab-983c-e8a1028fa11c,"アンパサンドアンパサンド（&amp;, ）は、並立助詞「…と…」を意味する記号である。"
1,beb8981c-a14b-4aab-983c-e8a1028fa11c,"ラテン語で「…と…」を表す接続詞 ""et"" の合字を起源とする。"
2,beb8981c-a14b-4aab-983c-e8a1028fa11c,"現代のフォントでも、Trebuchet MS など一部のフォントでは、""et"" の合字である..."
3,beb8981c-a14b-4aab-983c-e8a1028fa11c,"語源.英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字（""A..."
4,beb8981c-a14b-4aab-983c-e8a1028fa11c,"また、アルファベットの最後に、27番目の文字のように ""&amp;"" を加えることも広く行わ..."
...,...,...
0,bf7e5881-377c-4604-9ee8-b3b8f1564871,馬凡舒馬 凡舒（ば ぼんじょ、1993年11月6日 - ）は、中華人民共和国の司会者。
1,bf7e5881-377c-4604-9ee8-b3b8f1564871,経歴.1993年11月6日、黒竜江省ハルビン市で生まれる。
2,bf7e5881-377c-4604-9ee8-b3b8f1564871,2012年、中国伝媒大学に入学。
3,bf7e5881-377c-4604-9ee8-b3b8f1564871,2021年7月16日 - 現在、中央広播電視総台の司会を務めた。


In [77]:
our_dataframe_wiki.to_csv("JapaneseWikiArticles.csv", index=False)

We would need to merge sentences when dealing with TF-IDF!!! But sentences will be used for second part(graph).

## Getting content by articles

We got sentences from wiki, but it's better to work with articles(although sentences will be needed later)

# Cleaning our data

In [3]:
new_data = pd.read_csv('JapaneseWikiArticles.csv')

In [4]:
new_data

,article_name,content
0,beb8981c-a14b-4aab-983c-e8a1028fa11c,"アンパサンドアンパサンド（&amp;, ）は、並立助詞「…と…」を意味する記号である。"
1,beb8981c-a14b-4aab-983c-e8a1028fa11c,"ラテン語で「…と…」を表す接続詞 ""et"" の合字を起源とする。"
2,beb8981c-a14b-4aab-983c-e8a1028fa11c,"現代のフォントでも、Trebuchet MS など一部のフォントでは、""et"" の合字である..."
3,beb8981c-a14b-4aab-983c-e8a1028fa11c,"語源.英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字（""A..."
4,beb8981c-a14b-4aab-983c-e8a1028fa11c,"また、アルファベットの最後に、27番目の文字のように ""&amp;"" を加えることも広く行わ..."
...,...,...
21943871,bf7e5881-377c-4604-9ee8-b3b8f1564871,馬凡舒馬 凡舒（ば ぼんじょ、1993年11月6日 - ）は、中華人民共和国の司会者。
21943872,bf7e5881-377c-4604-9ee8-b3b8f1564871,経歴.1993年11月6日、黒竜江省ハルビン市で生まれる。
21943873,bf7e5881-377c-4604-9ee8-b3b8f1564871,2012年、中国伝媒大学に入学。
21943874,bf7e5881-377c-4604-9ee8-b3b8f1564871,2021年7月16日 - 現在、中央広播電視総台の司会を務めた。


No we need to group by page and then when TF-IDF, to merge sentences. In total we will have 2 csv: for sentences and for documents. Then let's merge our data(that is categorized by setntences).

In [ ]:
def grouping_data(data: pd.DataFrame) -> pd.DataFrame:
    # getting unique uuid(name or id)
    page_id = pd.DataFrame(data['article_name'].unique(), columns=['article_name'])
    page_id = page_id.reset_index()
    # and resetting index so that we can recieve article number
    page_id = page_id.rename(columns={'index': 'pg_index'})
    # now that we have unique ids, we can merge all sentences in one article. First we numerate sentences by pages
    # Also we are getting rid of article_name
    data = data.merge(page_id, on='article_name', how='left')[['pg_index', 'content']]
    # Joining all the sentences using groupby
    data_new = data.astype({'content': str}).groupby(by='pg_index').agg({'content': ' '.join})
    
    return data_new

In [5]:
def grouping_data_new(path_to_data: str) -> pd.DataFrame:
    data = pd.read_csv(path_to_data)
    # getting unique uuid(name or id)
    page_id = pd.DataFrame(data['article_name'].unique(), columns=['article_name'])
    page_id = page_id.reset_index()
    # and resetting index so that we can recieve article number
    page_id = page_id.rename(columns={'index': 'pg_index'})
    # now that we have unique ids, we can merge all sentences in one article. First we numerate sentences by pages
    # Also we are getting rid of article_name
    data = data.merge(page_id, on='article_name', how='left')[['pg_index', 'content']]
    # Joining all the sentences using groupby
    data_new = data.astype({'content': str}).groupby(by='pg_index')
    data_new = data_new.agg({'content': ' '.join})
    
    return data_new

In [52]:
part_of_data = new_data[:100000]
part_of_data

,article_name,content
0,beb8981c-a14b-4aab-983c-e8a1028fa11c,"アンパサンドアンパサンド（&amp;, ）は、並立助詞「…と…」を意味する記号である。"
1,beb8981c-a14b-4aab-983c-e8a1028fa11c,"ラテン語で「…と…」を表す接続詞 ""et"" の合字を起源とする。"
2,beb8981c-a14b-4aab-983c-e8a1028fa11c,"現代のフォントでも、Trebuchet MS など一部のフォントでは、""et"" の合字である..."
3,beb8981c-a14b-4aab-983c-e8a1028fa11c,"語源.英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字（""A..."
4,beb8981c-a14b-4aab-983c-e8a1028fa11c,"また、アルファベットの最後に、27番目の文字のように ""&amp;"" を加えることも広く行わ..."
...,...,...
99995,4d03f569-8cd3-4362-befe-06f8bfd4e616,そのために、天皇を中心として国民を一つにまとめる反面、議会に力を持たせ、バランスの取れた憲法...
99996,4d03f569-8cd3-4362-befe-06f8bfd4e616,憲法の起草は、夏島（現在の神奈川県横須賀市夏島町）の伊藤博文別荘を本拠に、1887年（明治2...
99997,4d03f569-8cd3-4362-befe-06f8bfd4e616,伊藤の別荘は手狭だったことから、事務所として料理旅館の「東屋」（現在の神奈川県横浜市金沢区）...
99998,4d03f569-8cd3-4362-befe-06f8bfd4e616,しかし、8月6日、伊藤らが横浜へ娯遊中に泥棒が入り、草案の入った鞄が盗難にあったことから、そ...


In [59]:
grouped_data = grouping_data(part_of_data)
# so that id and pg_index aren't mixed, it's a variable, not and id
grouped_data = grouped_data.reset_index()

In [61]:
grouped_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1480 entries, 0 to 1479
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pg_index  1480 non-null   int64 
 1   content   1480 non-null   object
dtypes: int64(1), object(1)
memory usage: 23.2+ KB


In [ ]:
grouped_all = grouping_data(new_data)

In [ ]:
grouped_all

In [6]:
please_groupby = grouping_data_new('JapaneseWikiArticles.csv')

In [7]:
please_groupby

,content
pg_index,
0,"アンパサンドアンパサンド（&amp;, ）は、並立助詞「…と…」を意味する記号である。 ラテ..."
1,言語言語（げんご）は、広辞苑や大辞泉には次のように解説されている。 『日本大百科事典』では、...
2,日本語 日本語（にほんご、にっぽんご、）は、日本国内や、かつての日本領だった国、そして日本人...
3,地理学地理学（ちりがく、、、伊：geografia、）は、。 地域や空間、場所、自然環境とい...
4,国の一覧国の一覧（くにのいちらん）は、世界の独立国の一覧。 対象.国際法上国家と言えるか否か...
...,...
1246611,藤白台藤白台（ふじしろだい）は大阪府吹田市にある地名。 千里ニュータウンにある住区である。 ...
1246612,林家たま平の青春スクラム『林家たま平の青春スクラム』（はやしやたまへいのせいしゅんすくらむ）...
1246613,"いつかあの海で『いつかあの海で』（いつかあのうみで、英：""Fleet Girls Colle..."


In [8]:
reseted_articles = please_groupby.reset_index()

In [9]:
reseted_articles

,pg_index,content
0,0,"アンパサンドアンパサンド（&amp;, ）は、並立助詞「…と…」を意味する記号である。 ラテ..."
1,1,言語言語（げんご）は、広辞苑や大辞泉には次のように解説されている。 『日本大百科事典』では、...
2,2,日本語 日本語（にほんご、にっぽんご、）は、日本国内や、かつての日本領だった国、そして日本人...
3,3,地理学地理学（ちりがく、、、伊：geografia、）は、。 地域や空間、場所、自然環境とい...
4,4,国の一覧国の一覧（くにのいちらん）は、世界の独立国の一覧。 対象.国際法上国家と言えるか否か...
...,...,...
1246611,1246611,藤白台藤白台（ふじしろだい）は大阪府吹田市にある地名。 千里ニュータウンにある住区である。 ...
1246612,1246612,林家たま平の青春スクラム『林家たま平の青春スクラム』（はやしやたまへいのせいしゅんすくらむ）...
1246613,1246613,"いつかあの海で『いつかあの海で』（いつかあのうみで、英：""Fleet Girls Colle..."
1246614,1246614,ヴィクトル・ジュール・ルバスールヴィクトル・ジュール・ルバスール () はフランスの地理学者...


In [10]:
reseted_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1246616 entries, 0 to 1246615
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   pg_index  1246616 non-null  int64 
 1   content   1246616 non-null  object
dtypes: int64(1), object(1)
memory usage: 19.0+ MB


We don't have any null object => let's save our file

In [11]:
reseted_articles.to_csv("GroupedArticles.csv", index=False)

In [12]:
checker = pd.read_csv("GroupedArticles.csv")

In [13]:
checker

,pg_index,content
0,0,"アンパサンドアンパサンド（&amp;, ）は、並立助詞「…と…」を意味する記号である。 ラテ..."
1,1,言語言語（げんご）は、広辞苑や大辞泉には次のように解説されている。 『日本大百科事典』では、...
2,2,日本語 日本語（にほんご、にっぽんご、）は、日本国内や、かつての日本領だった国、そして日本人...
3,3,地理学地理学（ちりがく、、、伊：geografia、）は、。 地域や空間、場所、自然環境とい...
4,4,国の一覧国の一覧（くにのいちらん）は、世界の独立国の一覧。 対象.国際法上国家と言えるか否か...
...,...,...
1246611,1246611,藤白台藤白台（ふじしろだい）は大阪府吹田市にある地名。 千里ニュータウンにある住区である。 ...
1246612,1246612,林家たま平の青春スクラム『林家たま平の青春スクラム』（はやしやたまへいのせいしゅんすくらむ）...
1246613,1246613,"いつかあの海で『いつかあの海で』（いつかあのうみで、英：""Fleet Girls Colle..."
1246614,1246614,ヴィクトル・ジュール・ルバスールヴィクトル・ジュール・ルバスール () はフランスの地理学者...
